In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn

In [ ]:
audiData = pd.read_csv("../input/used-car-dataset-ford-and-mercedes/audi.csv")

In [ ]:
audiData.head()

In [ ]:
audiModel = audiData.iloc[:,0:1].values

In [ ]:
from sklearn import preprocessing

In [ ]:
le = preprocessing.LabelEncoder()

In [ ]:
audiModel[:,0]=le.fit_transform(audiData.iloc[:,0])

In [ ]:
ohe = preprocessing.OneHotEncoder()
audiModel = ohe.fit_transform(audiModel).toarray()

In [ ]:
audiColumns = audiData["model"].unique()

In [ ]:
audiModel = pd.DataFrame(data=audiModel,index=range(10668),columns=audiColumns)

In [ ]:
audiModel

In [ ]:
newData = audiData.drop(["model"], axis = 1)

In [ ]:
audiData = pd.concat([audiModel, newData], axis = 1)

In [ ]:
audiData

In [ ]:
audiData.describe()

In [ ]:
audiData.isnull().sum()

In [ ]:
plt.figure(figsize = (7,5))
sbn.kdeplot(audiData["price"], fill=True)

In [ ]:
len(audiData) * 0.01

In [ ]:
cleanData = audiData.sort_values("price", ascending = False).iloc[100:]

In [ ]:
plt.figure(figsize = (7,5))
sbn.kdeplot(cleanData["price"], fill=True)

In [ ]:
audiData = cleanData

In [ ]:
y = audiData["price"].values
x = audiData.drop(["price", "transmission", "fuelType"], axis = 1).values

In [ ]:
y

In [ ]:
x

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 15)

In [ ]:
len(x_train)

In [ ]:
len(x_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
x_train.shape

In [ ]:
model = Sequential()

model.add(Dense(12, activation = "relu"))
model.add(Dense(12, activation = "relu"))
model.add(Dense(12, activation = "relu"))

model.add(Dense(1))

model.compile(optimizer = "adam", loss = "mse")
earlyStopping = EarlyStopping(monitor = "val_loss", mode = "min", verbose = 1, patience = 25)

In [ ]:
model.fit(x = x_train, y = y_train, epochs = 700, validation_data = (x_test, y_test), verbose = 1, batch_size = 250, callbacks = [earlyStopping,])

In [ ]:
lossData = pd.DataFrame(model.history.history)

In [ ]:
lossData.head()

In [ ]:
lossData.plot()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
predictedArr = model.predict(x_test)

In [ ]:
predictedArr

In [ ]:
mean_absolute_error(y_test, predictedArr)

In [ ]:
plt.scatter(y_test, predictedArr)
plt.plot(y_test, y_test,"r-*")

In [ ]:
realDf = pd.DataFrame(y_test, columns = ["Real Y"])

In [ ]:
realDf

In [ ]:
predictedArr = pd.Series(predictedArr.reshape(3171,))

In [ ]:
predictedArr

In [ ]:
predictedDf = pd.concat([realDf, predictedArr], axis = 1)

In [ ]:
predictedDf.columns = ["Real Price", "Predicted Price"]

In [ ]:
predictedDf